# Weak References - Part 1 of 2

https://docs.python.org/3/library/weakref.html

We work with "raw" weak referneces first, in part for conceptual clarity. But most use of `weakref` facilities in Python should not be through weak reference objects (`weakref.ref` and `weakref.proxy`).

Instead, one usually uses the higher level facilities: `weakref.finalize`, `weakref.WeakSet`, `weakref.WeakKeyDictionary`, and `weakref.WeakValueDictionary`.

Sometimes, when building linked data structures (that store data in nodes), one uses "raw" weak references directly to avoid unnecessary strong cycles. That may be the most "basic" direct use of weak references in Python where it is totally reasonable to use them, and not reasonable to replace most uses with higher-level facilities.

You should reach for the higher level facilities first, in most cases.

In [1]:
import gc
import weakref

## Weak reference basics

Instances of unslotted classes are weak-referenceable:

In [2]:
class C: pass

In [3]:
c = C()

In [4]:
repr(c)

'<__main__.C object at 0x0000018FB76FA1A0>'

In [5]:
print(c)

In [6]:
r = weakref.ref(c)

A reference's *referent* is the object it refers to. The `repr` of a `weakref.ref` shows whether its referent is alive or dead. Of course, this may change at any time:

In [7]:
r

<weakref at 0x0000018FB77267A0; to 'C' at 0x0000018FB76FA1A0>

Call a weak reference to access its referent. If the referent is dead, `None` is returned instead:

In [8]:
print(r())

In [9]:
r() is c

True

In [10]:
del c

In [11]:
print(r())

None


The `None` object itself is not weak-referenceable. One consequence of this is that there is no ambiguity (no semipredicate problem) in the meaning of the object returned by calling a `weakref.ref`:

In [12]:
n = None

In [13]:
n

In [14]:
rn = weakref.ref(n)

TypeError: cannot create weak reference to 'NoneType' object

`int`s are likewise not weak-referenceable:

In [15]:
rone = weakref.ref(1)

TypeError: cannot create weak reference to 'int' object

## Slotted classes and weak references

Instances of slotted classes are by default (and including in the case of empty slots) not weak-referenceable:

In [16]:
class CSlots: 
    __slots__ = ()

In [17]:
weakref.ref(CSlots())

TypeError: cannot create weak reference to 'CSlots' object

As with instance dictionaries, inheriting from a class whose instances are weak-referenceable gives a class whose instances are weak-referenceable regardless of whether the derived cass defines `__slots__` or what it lists in them.

However, the ability to be weakly referenced should not be confused with having an instance dictionary. On rare occasion, one may want a slotted class whose instances have instance dictionaries. Instances of such a class are not automatically weak-referenceable:

In [18]:
class DSlots: 
    __slots__ = ('__dict__',)

In [19]:
weakref.ref(DSlots())

TypeError: cannot create weak reference to 'DSlots' object

In [20]:
DSlots().__dict__

{}

## The `__weakref__` slot

To ensure instances of a class are weak referenceable, instances must have a `__weakref__` slot. This exists automatically if the class is unslotted, as does `__dict__`. If a class is slotted and all its direct and indirect base classes are slotted, then `__weakref__` must be defined to ensure weak-referenceability (and in CPython, in the absence of it, instances are guaranteed *not* to be weakly referenceable):

In [21]:
class WRSlots: 
    __slots__ = ('__weakref__',)

In [22]:
wr = WRSlots()

In [23]:
weakref.ref(wr)

<weakref at 0x0000018FB92CA750; to 'WRSlots' at 0x0000018FB76FB1C0>

In [24]:
weakref.ref(WRSlots())

<weakref at 0x0000018FB92CAED0; dead>

Note that the rules for slot duplication are not relaxed for `__weakref__`. If a base class is slotted and names `__weakref__` in its `__slots__`, then a directly or indirectly derived class *must not* also do so. If it does, then like with any other slot dupliacted from base to derived class, this is undefined behavior.

## Callbacks

Weak references can be created with callbacks. Assuming the weak reference itself still exists when the referent dies, the callback is called. The weak reference itself, and *not* the referent, is passed to the callback. The referent is, ideally, already dead and collected by the time the callback runs (though we will see that is not always the case). Whether or not the referent still exists, the weak reference sees it as dead at the time the callback is called. Consequently, weak reference callbacks do not resurrect objects. This is a very nice thing.

In [25]:
c2 = C()

In [26]:
wr = weakref.ref(c2, lambda x: print(f"{x}'s referent is dead!"))

In [27]:
wr

<weakref at 0x0000018FB929BE20; to 'C' at 0x0000018FB76FBAC0>

In [28]:
del c2

<weakref at 0x0000018FB929BE20; dead>'s referent is dead!


## Unhandled exceptions in callbacks

An unhandled exception in a callback, or that occurs *in attempting to call the callback*, cannot be caught. It is reported on standard error, in the same way as an unhandled exception in a `__del__` method.

In [29]:
wr

<weakref at 0x0000018FB929BE20; dead>

In [30]:
wr2 = weakref.ref(C(), lambda: print("The referent is dead!"))

Exception ignored in: <function <lambda> at 0x0000018FB92A7910>
Traceback (most recent call last):
  File "C:\Users\ek\AppData\Local\Temp\ipykernel_3388\3704447829.py", line 1, in <module>
TypeError: <lambda>() takes 0 positional arguments but 1 was given


In [31]:
wr2

<weakref at 0x0000018FB92CD350; dead>

## Cycles

Weak reference callbacks are called when an object dies, even if the object has to be collected by the cyclic garbage collector.

In [32]:
cycle = C()

In [33]:
cycle.x = cycle

In [34]:
wrcycle = weakref.ref(cycle, lambda x: print("Referent (cycle) has died!"))

In [35]:
del cycle

In [36]:
gc.collect()

Referent (cycle) has died!


1122

## `weakref.ref` equality comparison

In [37]:
class Box: 
    
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value

In [38]:
x = Box(1)
y = Box(1)

In [39]:
x == y

True

In [40]:
print(x)

Box(1)


### Calling `weakref.ref` may give an existing weak reference instance

In [41]:
r1 = weakref.ref(x)
r2 = weakref.ref(x)

In [42]:
r1 == r2

True

In [43]:
r1 is r2  # Not guaranteed.

True

In [44]:
r3 = weakref.ref(x, lambda _: print("Dead box"))

In [45]:
r3 is r1  # Guaranteed.

False

### Equality with live referents

If the referents are both alive and equal to each other, then the weak references to them are equal.

In [46]:
r3 == r1

True

In [47]:
s = weakref.ref(y)

In [48]:
r1 == s  # Weak references compare equal when their referents compare equal. 

True

Note that, because `_Box` instances are mutable and non-hashable, `weakref.ref`s to them are likewise unhashable.

In [49]:
hash(s)

TypeError: unhashable type: 'Box'

When referents stop being equal because of a change in state, weak refernces to them likewise stop being equal:

In [50]:
z = Box(1)

In [51]:
t = weakref.ref(z)

In [52]:
r1 == s == t

True

In [53]:
z.value = 2

In [54]:
print(z)

Box(2)


In [55]:
s == t

False

### Equality with dead referents

If the referents are dead, then weak references to them are only equal if they are the same `weakref.ref` object. Even separate `weakref.ref` objects that refer to the *same* referent become unequal when the referent dies.

In [56]:
del x

Dead box


In [57]:
r1 == r2

True

In [58]:
r1 is r2  # Because they are the same weak reference... which was NOT guaranteed!

True

In [59]:
r1 == r3  # Different weak references to the same dead referent.

False

### Weak references to live objects can *become* equal

In [60]:
print(y)

Box(1)


In [61]:
print(z)

Box(2)


In [62]:
s == t

False

In [63]:
z.value = 1

In [64]:
s == t

True

### Weak references to dead objects can become equal to themselves

This happens in the pathological case of NaN-like objects that are not equal to themselves.

In [65]:
import math

In [66]:
nbox = Box(math.nan)

In [67]:
nbox == nbox

False

In [68]:
nbref = weakref.ref(nbox)

In [69]:
nbref == nbref

False

In [70]:
del nbox

In [71]:
nbref == nbref

True

## `weakref.ref` hashing

In [72]:
class LockBox: 
    
    def __init__(self, value): 
        self._value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value

    def __hash__(self): 
        return hash(self.value)

    @property
    def value(self): 
        return self._value

In [73]:
lb = LockBox(1)

In [74]:
print(lb)

LockBox(1)


In [75]:
g = weakref.ref(lb)

In [76]:
hash(g)

1

In [77]:
del lb

In [78]:
g

<weakref at 0x0000018FB9226C00; dead>

The `weakref.ref` object cached the hash code and continues to return it:

In [79]:
hash(g)

1

Of course, that doesn't work if the hash code was never computed through the weak reference during the object's lifetime:

In [80]:
lb2 = LockBox(2)

In [81]:
h = weakref.ref(lb2)

In [82]:
del lb2

In [83]:
hash(h)

TypeError: weak object has gone away

## Weak reference callbacks and `__del__` methods

In [84]:
class SithHolocron: 
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value
    
    def __del__(self): 
        print(f"I see you are attempting to destroy {self!r}!")

### Without cycles

Normally, the `__del__` method is called before the weak reference callback:

In [85]:
sh = SithHolocron('ancient sith secrets')

In [86]:
sithweak = weakref.ref(sh, lambda x: print(f"{x!r}: Holocron destroyed!"))

In [87]:
del sh

I see you are attempting to destroy SithHolocron('ancient sith secrets')!
<weakref at 0x0000018FB884A110; dead>: Holocron destroyed!


This makes sense, since, ideally, the `__del__` method is called just before the object is destroyed, while, ideally, the weak reference callback is called just after the object is destroyed.

### With cycles

The above behavior is not guaranteed, and in practice, varies by implementation in the presence of cycles.

In *CPython*, the behavior demonstrated here is guaranteed.

In [88]:
sh2 = SithHolocron('modern sith secrets')

In [89]:
sh2.backup = sh2

In [90]:
modernsithweak = weakref.ref(sh2, lambda x: print(f"{x!r}: Holocron destroyed!"))

In [91]:
del sh2

In [92]:
gc.collect()  # 'dead' means not reachable

<weakref at 0x0000018FB92BBEC0; dead>: Holocron destroyed!
I see you are attempting to destroy SithHolocron('modern sith secrets')!


960

Unintuitively, the weak refernece callback was called before the `__del__` method! This is because CPython's cyclic garbage collector sees to it that weak references are invalidated, and their callbacks call, before it proceeds to run finalizer methods (`__del__` methods).

## Weak reference callbacks and `__del__` methods: Resurrection

In [93]:
ringsofpower = []

In [94]:
class RingOfPower: 
    def __init__(self, value): 
        self.value = value
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.value!r})'
    
    def __eq__(self, other): 
        if not isinstance(other, type(self)): 
            return NotImplemented
        return self.value == other.value
    
    def __del__(self): 
        print(f"I see you are attempting to destroy {self!r}!")
        ringsofpower.append(self)        

### Without cycles

In [95]:
onering = RingOfPower('one ring to rule them all ...')

In [96]:
weakring = weakref.ref(onering, lambda x: print(f"{x!r} has casted the ring into the fire!"))

In [97]:
del onering

I see you are attempting to destroy RingOfPower('one ring to rule them all ...')!


In [98]:
weakring

<weakref at 0x0000018FB92B95D0; to 'RingOfPower' at 0x0000018FB881E5F0>

In [99]:
print(ringsofpower)

[RingOfPower('one ring to rule them all ...')]


In [100]:
ringsofpower.clear()

<weakref at 0x0000018FB92B95D0; dead> has casted the ring into the fire!


### With cycles

In [101]:
wkring = RingOfPower("The Witch King's ring")

In [102]:
wkring.backup = wkring 

In [103]:
wkref = weakref.ref(wkring, lambda x: print(f"{x!r} (not a man) has killed the Witch King."))

In [104]:
del wkring

In [105]:
gc.collect()

<weakref at 0x0000018FB7726CF0; dead> (not a man) has killed the Witch King.
I see you are attempting to destroy RingOfPower("The Witch King's ring")!


0

In [106]:
print(ringsofpower)  # BUT WE KILLED HIM!!! (Don't resurrect objects)

[RingOfPower("The Witch King's ring")]


## Lifetime of weak references

If you delete the weak reference before you delete the referent, the callback will not be called:

In [107]:
nlb = LockBox('The budget surplus')

In [108]:
lbwref = weakref.ref(nlb, lambda _: print("There goes the lockbox"))

In [109]:
del nlb

There goes the lockbox


In [110]:
nlb2 = LockBox('The budget deficit')

In [111]:
lb2wref = weakref.ref(nlb2, lambda _: print("There goes the lockbox!"))

In [112]:
del lb2wref

In [113]:
del nlb2

This motivates weakref finalizers.

## Weak reference finalizers

The lifetime of a weak referenece finalizer is tied to that of the referent. When using weakref finalizers, we don't directly get access to a weak reference object. But whether the finalizer object is strongly reachable or not, the finalizer ensures it remains alive as long as its referent, so the function passed when constructing the finalizer will be reliably called when the referent dies.

In [114]:
bigbox = Box('BIG!')

In [115]:
bigboxfin = weakref.finalize(bigbox, lambda: print("Destroyed a big box."))

In [116]:
del bigboxfin

In [117]:
del bigbox

Destroyed a big box.


### Weakref finalizer arguments

Unlike a weak reference callback, no weak reference object is automatically passed to the function registered with a finalizer. Instead, whatever positional and/or keyword arguments are passed to `weakref.finalize` after the function are stored, and they are passed to the function:

In [118]:
lockbox = LockBox('Hidden costs')

In [119]:
lockboxfin = weakref.finalize(lockbox, lambda x: print(f'There were... {x}.'), lockbox.value)

In [120]:
del lockbox

There were... Hidden costs.


### Beware: you can leak the finalizer and referent

If a weakref finalizer holds a strong reference to the object it is supposed to "finalize", then the object can never be collected. This leaks the object (and the finalizer):

In [121]:
lockbox2 = LockBox('Big hidden costs.')

In [122]:
lockbox2fin = weakref.finalize(lockbox2, lambda x: print(f'There were... {x.value}.'), lockbox2)

In [123]:
del lockbox2, lockbox2fin

Note that, even if the function we register has no parameters, and we pass no arguments to `weakref.finalize` after it, we can still write this kind of bug, if the function we pass *captures* a variable that refers to the object.

### What `weakref.finalize` returns

It has a useful `repr`, giving information like that of `weakref.ref`.

In [124]:
ccube = Box('heart')

In [125]:
ccubefin = weakref.finalize(ccube, lambda: print("DON'T TOSS ME IN THERE!"))

In [126]:
ccubefin

<finalize object at 0x18fb9166da0; for 'Box' at 0x18fb897cc40>

In [127]:
del ccube

DON'T TOSS ME IN THERE!


In [128]:
ccubefin

<finalize object at 0x18fb9166da0; dead>

### Using the finalizer object `weakref.finalize` returns

In [129]:
bcube = Box('blue')

In [130]:
bcubefin = weakref.finalize(bcube, lambda: print("Blue cube vaporized!"))

In [131]:
dir(bcubefin)

['_Info',
 '__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_dirty',
 '_exitfunc',
 '_index_iter',
 '_registered_with_atexit',
 '_registry',
 '_select_for_exit',
 '_shutdown',
 'alive',
 'atexit',
 'detach',
 'peek']

In [132]:
bcubefin()

Blue cube vaporized!


In [133]:
bcubefin

<finalize object at 0x18fb92a90c0; dead>

In [134]:
print(bcube)

Box('blue')


In [135]:
tup = (bcube,)

In [136]:
del tup[0]

TypeError: 'tuple' object doesn't support item deletion

## Using `weakref.finalize` to explore `itertools.tee`

This is the `itertools.tee` element lifetime exercise from `gencomp3.ipynb`, but with weak reference finalizers instead of `__del__`.

In [137]:
import itertools

In [138]:
def boxgen(): 
    for x in range(200): 
        box = Box(x) 
        weakref.finalize(box, lambda value: print(f"#{value} is being finalized"), box.value)
        yield box

In [139]:
x, y = itertools.tee(boxgen())

In [140]:
for _ in range(100): 
    b = next(y).value

In [141]:
for _ in range(50): 
    c = next(x).value

In [142]:
for _ in range(50): 
    c = next(x).value

#0 is being finalized
#1 is being finalized
#2 is being finalized
#3 is being finalized
#4 is being finalized
#5 is being finalized
#6 is being finalized
#7 is being finalized
#8 is being finalized
#9 is being finalized
#10 is being finalized
#11 is being finalized
#12 is being finalized
#13 is being finalized
#14 is being finalized
#15 is being finalized
#16 is being finalized
#17 is being finalized
#18 is being finalized
#19 is being finalized
#20 is being finalized
#21 is being finalized
#22 is being finalized
#23 is being finalized
#24 is being finalized
#25 is being finalized
#26 is being finalized
#27 is being finalized
#28 is being finalized
#29 is being finalized
#30 is being finalized
#31 is being finalized
#32 is being finalized
#33 is being finalized
#34 is being finalized
#35 is being finalized
#36 is being finalized
#37 is being finalized
#38 is being finalized
#39 is being finalized
#40 is being finalized
#41 is being finalized
#42 is being finalized
#43 is being finalize

In [143]:
del x, y

#57 is being finalized
#58 is being finalized
#59 is being finalized
#60 is being finalized
#61 is being finalized
#62 is being finalized
#63 is being finalized
#64 is being finalized
#65 is being finalized
#66 is being finalized
#67 is being finalized
#68 is being finalized
#69 is being finalized
#70 is being finalized
#71 is being finalized
#72 is being finalized
#73 is being finalized
#74 is being finalized
#75 is being finalized
#76 is being finalized
#77 is being finalized
#78 is being finalized
#79 is being finalized
#80 is being finalized
#81 is being finalized
#82 is being finalized
#83 is being finalized
#84 is being finalized
#85 is being finalized
#86 is being finalized
#87 is being finalized
#88 is being finalized
#89 is being finalized
#90 is being finalized
#91 is being finalized
#92 is being finalized
#93 is being finalized
#94 is being finalized
#95 is being finalized
#96 is being finalized
#97 is being finalized
#98 is being finalized
#99 is being finalized


## Weak sets

A weak set is a "set" that doesn't keep its elements alive. Behind the scenes, it holds them with weak references.

In [144]:
s = {LockBox(10), LockBox(20)}

In [145]:
s

{LockBox(10), LockBox(20)}

In [146]:
ws = weakref.WeakSet((LockBox(10), LockBox(20)))

In [147]:
ws

set()

In [148]:
repr(ws)

'set()'

In [149]:
supplies = {good: LockBox(good) for good in ('eggs', 'wheat', 'steel', 'clothes', 'iron')}

In [150]:
supplies

{'eggs': LockBox('eggs'),
 'wheat': LockBox('wheat'),
 'steel': LockBox('steel'),
 'clothes': LockBox('clothes'),
 'iron': LockBox('iron')}

In [151]:
tax_exempt = {supplies['steel'], supplies['iron']}

In [152]:
del supplies['steel']

In [153]:
supplies

{'eggs': LockBox('eggs'),
 'wheat': LockBox('wheat'),
 'clothes': LockBox('clothes'),
 'iron': LockBox('iron')}

In [154]:
tax_exempt

{LockBox('iron'), LockBox('steel')}

In [155]:
consumer_goods = weakref.WeakSet((supplies['eggs'], supplies['wheat'], supplies['clothes']))

In [156]:
consumer_goods

{<weakref at 0x0000018FB930C450; to 'LockBox' at 0x0000018FB8840D60>, <weakref at 0x0000018FB930C540; to 'LockBox' at 0x0000018FB8842620>, <weakref at 0x0000018FB930C400; to 'LockBox' at 0x0000018FB8842E60>}

In [157]:
del supplies['eggs']

In [158]:
consumer_goods

{<weakref at 0x0000018FB930C450; to 'LockBox' at 0x0000018FB8840D60>, <weakref at 0x0000018FB930C400; to 'LockBox' at 0x0000018FB8842E60>}

In [159]:
repr(set(consumer_goods))

"{LockBox('wheat'), LockBox('clothes')}"

In [160]:
del supplies['clothes']

In [161]:
repr(set(consumer_goods))

"{LockBox('wheat')}"

Weak sets (and other weak collections) use weak reference callbacks to removed weak references to elements after the elements have died.

## Weak-key dictionaries

In [162]:
meq = {unit: LockBox(unit) for unit in ('tanks', 'aircraft', 'guns', 'ships')}

If we don't need `meq`, then of course we don't have to bother with this. Non-contrived situations where it is reasonable to do this are when the keys are not necessarily lightweight objects that we reasonably pass around copies of.

In [163]:
usakit = weakref.WeakKeyDictionary({meq['tanks']: 'M4 Sherman', meq['aircraft']: 'P51 Mustang', meq['guns']: 'M1 Garand', meq['ships']: 'Liberty Ships'})

In [164]:
usakit

In [165]:
repr(dict(usakit))

"{LockBox('tanks'): 'M4 Sherman', LockBox('aircraft'): 'P51 Mustang', LockBox('guns'): 'M1 Garand', LockBox('ships'): 'Liberty Ships'}"

In [166]:
del meq['tanks']

In [167]:
repr(dict(usakit))

"{LockBox('aircraft'): 'P51 Mustang', LockBox('guns'): 'M1 Garand', LockBox('ships'): 'Liberty Ships'}"

An example of `WeakKeyDictionary` in this project is `test_context._AttributeSpy`, which uses `_attribute_spy_histories`.

Another way to achieve the goal there would be to define `__del__` in `_AttributeSpy` and have it remove an entry from `_attribute_spy_histories` and have `_attribute_spy_histories` just be a `dict`. But that has the problems of `__del__`.

We can think of a `WeakSet` as a specialized collection that we can consider using, in some cases, instead of a `WeakKeyDictionary`. A `WeakSet` stores answers to yes-no questions; a `WeakKeyDictionary` stories any kind of answer.

## Weak-value dictionaries

In [168]:
weakvaltest = weakref.WeakValueDictionary({1: Box(1)})

In [169]:
weakvaltest

In [170]:
repr(dict(weakvaltest))

'{}'

In [171]:
ussrkit = {model: LockBox(model) for model in ('T-34', 'IL-2', 'PPSH', 'red-october')}

In [172]:
ww2kit = weakref.WeakValueDictionary({'tank': ussrkit['T-34'], 'CAS': ussrkit['IL-2'], 'submgun': ussrkit['PPSH'], 'sub': ussrkit['red-october']})

In [173]:
import pprint

In [174]:
pprint.pp(ussrkit)

{'T-34': LockBox('T-34'),
 'IL-2': LockBox('IL-2'),
 'PPSH': LockBox('PPSH'),
 'red-october': LockBox('red-october')}


In [175]:
pprint.pp(dict(ww2kit))

{'tank': LockBox('T-34'),
 'CAS': LockBox('IL-2'),
 'submgun': LockBox('PPSH'),
 'sub': LockBox('red-october')}


In [176]:
del ussrkit['red-october']

In [177]:
pprint.pp(dict(ww2kit))

{'tank': LockBox('T-34'), 'CAS': LockBox('IL-2'), 'submgun': LockBox('PPSH')}


Weak-value dictionaries are often used for caching.

### `WeakValueDictionary` holds its keys strongly

Another example, related to the above code, is to make a weak inverse of a (strong) dictionary, where A -> B keeps the A objects alive (and the B ones), but going from B -> A does not keep the A objects alive, so removing an entry from the A -> B mapping removes the corresponding entry from the B -> A mapping:

In [178]:
ab = {LockBox('a1'): LockBox('b1'), LockBox('a2'): LockBox('b2'), LockBox('a3'): LockBox('b3')}

In [179]:
pprint.pp(ab)

{LockBox('a1'): LockBox('b1'),
 LockBox('a2'): LockBox('b2'),
 LockBox('a3'): LockBox('b3')}


In [180]:
ba = weakref.WeakValueDictionary({b: a for a, b in ab.items()})

In [181]:
pprint.pp(dict(ba))

{LockBox('b1'): LockBox('a1'),
 LockBox('b2'): LockBox('a2'),
 LockBox('b3'): LockBox('a3')}


In [182]:
a2 = ba[LockBox('b2')]  # Hold the reference.
repr(a2)

"LockBox('a2')"

In [183]:
del ab[LockBox('a1')]

In [184]:
pprint.pp(ab)

{LockBox('a2'): LockBox('b2'), LockBox('a3'): LockBox('b3')}


In [185]:
pprint.pp(dict(ba))

{LockBox('b2'): LockBox('a2'), LockBox('b3'): LockBox('a3')}


In [186]:
b3 = ab[LockBox('a3')]  # Hold the reference.
repr(b3)

"LockBox('b3')"

Outside our mappings, we have strong references `a2` and `b3`.

In [187]:
del ab[LockBox('a2')]

In [188]:
repr(ab)

"{LockBox('a3'): LockBox('b3')}"

In [189]:
repr(dict(ba))

"{LockBox('b2'): LockBox('a2'), LockBox('b3'): LockBox('a3')}"

In [190]:
del ab[LockBox('a3')]

In [191]:
repr(ab)

'{}'

In [192]:
repr(dict(ba))

"{LockBox('b2'): LockBox('a2')}"

In [193]:
b3

LockBox('b3')